# BiVital Basic Example
This notebook demonstrates how to use the BiVital library.

It covers how to load, explore, print and plot the data.


## Import the processing lib from bivtial and set up logging 

In [ ]:
from bivital import processing as bvl
from tkinter import filedialog, Tk
import logging

#Set up logging
logging.basicConfig(encoding='utf-8', level=logging.INFO)

## Choose the project directory for loading

In [ ]:
#Choose the example project data or choose your own directory
# 0 = example data
# 1 = add your path 
# else = Choose Directory --> This will open a file dialog for you to choose the directory, check the opened windows in the background

Data_to_use = 0

if Data_to_use == 0:
    project_directory = bvl.path_to_example_data()
elif Data_to_use == 1:
    project_directory = "Path/to/your/BiVital/Project"  # Replace with the path to your data
else:
    root = Tk()
    root.withdraw()
    project_directory = filedialog.askdirectory()
    print(f"Selected directory: {project_directory}")

## Load the data

In [ ]:
project_data = bvl.ProjectData(project_directory)

## Explore the project structure

---
### Architecture Diagram

```plaintext
ProjectData
└── SeriesA
    ├── datasets
    │   ├── Dataset1
    │   │   ├── df: DataFrame
    │   │   └── metadata: dict
    │   └── Dataset2
    └── summary
└── SeriesB
```

### Hierarchy and Nesting

| Level    | Key            | Type              | Description                  |
|----------|----------------|-------------------|------------------------------|
| Series   | `"SeriesA"`    | `dict`            | Groups datasets              |
| Dataset  | `"Dataset1"`   | `dict`            | One CSV/recording            |
| DataFrame| `"df"`         | `pandas.DataFrame`| Time-indexed data            |
| Metadata | `"metadata"`   | `dict`            | CSV header info              |


### Data Access (`__getitem__`)

Everything can be accessed by the Index aswell

| Expression                            | Returns                               |
|---------------------------------------|----------------------------------------|
| `project_data["SeriesA"]`              | All datasets in the series             |
| `project_data["SeriesA", y]`        | DataFrame of dataset                   |
| `project_data["SeriesA", y, z]` | Feature column (Series)               |
| `project_data["SeriesA", y, z,  42]` | Single value or time lookup        |
| `project_data.metadata[x, y]`         | Metadata for dataset                   |
| `project_data.metadata[x, y, "mac"]` | Specific metadata field               |


### Print the project data (`.print()`)

The `.print()`- function is a custom print for the library.

Use the `region[]` parameter to access the diffrent levels.
- `region`: Data for processing. [Default = whole project]
    - `[]`: whole project
    - `[series, dataset, column, row]`: specific data
    - `[slice(0,2), slice(0,2)]`: more than one series or data

---


Look at the Project Overview:

In [ ]:
project_data.print(region=[]) # Test the diffrent levels of the print function - [0], [0,0], [0,0,'ECG'], [0,0,'ECG',0]

### Access a Series

In [ ]:
print(project_data[0]) # All datasets in series 0

### Access a Dataset

In [ ]:
print(project_data[0, 0])  # First dataset (as DataFrame)

### Access specific Feature columns

In [ ]:
print(project_data[0, 0, ["Heartrate", "ECG" ]])

## Explore the Metadata

In [ ]:
print(project_data.metadata) # Project metadata

In [ ]:
print(project_data.metadata[0])  # Series metadata

In [ ]:
print(project_data.metadata[0, 0])  # Dataset metadata

In [ ]:
print(project_data.metadata[0, 0, 0])  # Feature metadata

## Plot the ECG and Heartrate samples from 200 to 500

for the plot the pandas.plot() function is used, for more Options follow the link https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html

In [ ]:
import matplotlib.pyplot as plt

# Create two vertically stacked subplots (2 rows, 1 column)
fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Plot ECG data on the first subplot
project_data[0, 0, "ECG", 200:500].dropna().plot(
    ax=axes[0],                 # Use the first Axes object (top subplot)
    grid=True,                  # Show grid lines for better readability
    title="ECG Example Data",   # Set plot title
    xlabel="",                  # Leave x-axis label empty (set it on the bottom subplot)
    ylabel="Amplitude"          # Label for the y-axis
)
axes[0].tick_params(axis='x', rotation=45)  # Rotate x-axis tick labels to avoid overlap

# Plot Heartrate data on the second subplot
project_data[0, 0, "Heartrate", 200:500].dropna().plot(
    ax=axes[1],                     # Use the second Axes object (bottom subplot)
    grid=True,                      # Show grid lines
    title="Heartrate Example Data", # Set plot title
    xlabel="Time",                  # Label for the x-axis
    ylabel="BPM"                    # Label for the y-axis (Beats Per Minute)
)
axes[1].tick_params(axis='x', rotation=45)  # Rotate x-axis tick labels

# Automatically adjust subplot layout to prevent overlap
plt.tight_layout()

# Display the final combined figure
plt.show()
